In [2]:
class HyperoptObjective(object):
    def __init__(self, X, y, model, const_params, fit_params, cv_splitter, cv_scoring):
        self.evaluated_count = 0
        self.X = X
        self.y = y
        self.model = model
        self.constant_params = const_params
        self.fit_params = fit_params
        self.cv_splitter = cv_splitter
        self.cv_scoring = cv_scoring
        
    '''
    The way that HyperOpt fmin function works, is that on each evaluation 
    it calls given objective function. 
    Since we decided to declare our objective as class instead of a function,
    we will implement the evaluation logic inside the __call__ method.
    '''
    def __call__(self, hyper_params):
        
        cv_result = cross_validate(
            self.model(**hyper_params, **self.constant_params),
            self.X, 
            self.y, 
            scoring=self.cv_scoring,
            cv=self.cv_splitter,
            fit_params=self.fit_params,
            return_train_score=True
        )
        
        '''
        We're going to calculate final iteration score as mean 
        of the scores from all CV folds.
        This choice is somewhat arbitrary, so it might be worth to consider
        another aggregate function, such as np.min or np.max.
        '''
        mean_cv_result = np.mean(cv_result['test_score'])

        self.evaluated_count += 1

        '''
        Hyperopt always tries to minimize loss. We will check _sign parametter attached 
        to scorer function by make_scorer (it is inferred from greater_is_better parameter). 
        If _sign is positive, we will flip mean_cv_result sign to convert score to loss. 
        '''
        if self.cv_scoring._sign > 0:
            mean_cv_result = -mean_cv_result 
        
        return mean_cv_result

In [ ]:
object = 